# What it is

A script that takes an excel xlsx file containing the url to check along with expected meta tags that a user wants to check. The tool outputs whether the tags have been set at all, incorrectly, or correctly.

The current meta tags that can be checked are:
* Meta Title
* Meta Description

# Before Running All Cells

Check the the xlsx file containing canonicals that you want to check follows the correct format. To see the expected format, view the xlsx file "Example Structure" under the Inputs folder. Then, place the xlsx file in the Check-Meta-Tags > Inputs folder.

Finally, enter file input below under [User Input](#user_input)

# How to Run
On top select Cell -> Run All

<a id='user_input'></a>

## User Input

Enter information below before running the cells.

In [1]:
# User input data
# Currently the fields are set to be 

META_TAGS_WORKBOOK = 'Example.xlsx'
META_TAGS_WORKBOOK_SHEET = 'MetaTagsSheet'


## Import and Constant

Cells in this section import libraries, define where the ouputted file will go, and load the file the user wants to use to check canonicals.

In [2]:
# Imports and constants
import urllib.request as request
import pandas as pd
import re
import xlrd
from xlutils.copy import copy
from datetime import datetime
import requests
import urllib.request
import urllib
import pandas as pd
from bs4 import BeautifulSoup

META_TAGS_INPUT_FOLDER = 'Inputs/'
META_TAGS_OUTPUT_FOLDER = 'Results/'

META_TAGS_INPUT_WORKBOOK_PATH = META_TAGS_INPUT_FOLDER + META_TAGS_WORKBOOK

to_check = xlrd.open_workbook(META_TAGS_INPUT_WORKBOOK_PATH)
to_check_sheet = to_check.sheet_by_name(META_TAGS_WORKBOOK_SHEET)

check_wb = copy(to_check) 
check_sheet = check_wb.get_sheet(META_TAGS_WORKBOOK_SHEET)

In [3]:
def return_tag(url, tag):
    ''' This function parses the html of the parameter url and returns the url of a canonical if set.'''
    f = urllib.request.urlopen(url)
    soup = BeautifulSoup(f.read(), 'html.parser')
    
    # Find all html elements that are of type link. Then, for each of these elements, if it is a canonical
    # link, set this to be the canonical and break out of the loop. There can only be one canonical set so no need
    # to check the rest of the links.
    result = "None set"
    if tag == "title":
        for link in soup.find(tag):
            result = link
    else:
        for link in soup.findAll('meta'):
            if link.get('name') == tag:
                result = link.get('content')
    return result
    
def add_https_if_none(url):
    '''Adds the full url path if none was defined on the input file.
    This assumes that the url should start with https. If it is still an http site, this will likely be the source
    of any issues. In that case, the user should explicitly define their urls to be http:// in the inputted file.'''
    if url.startswith("www"):
        return "https://" + url
    else: return url

# Testing

The cells below are a check to make sure that the tool is working correctly. If one of these fails, and the canonical checker still runs, outputted file may be incorrect. Reach out or trouble shoot based on the outputted error.

When selecting 'Run All Cells', if one of these tests fails, the code will stop running at this cell. If you want to continue, you can select the 'Actual Check' cell and continue by running that, but it's highly advised against.

In [4]:
from bs4 import BeautifulSoup
def test_return_tag(url, tag, actual):
    '''Test for checking that the canonical parser is working correctly.
    If this returns a warning, first check that the passed in url actually has the redirect.'''
    result = return_tag(url, tag)
    if result == actual:
        return True
    else:
        print("Error when parsing")
        return sys.exit(result)

print(test_return_tag("https://www.masterlock.com/business-use/product/A1266NBLK",
                      "title", 'Model No. A1266NBLK | Master Lock'))
print(test_return_tag("https://www.masterlock.com/business-use/product/A1266NBLK",
                "description", 
                "The American Lock A1266NBLK Solid Aluminum Padlock offers customization options to help fit your security needs, including keying, laser engraving and shackle options. Learn more.")) 

True
True


## Actual Check

Now on to applying the logic.

In [21]:
# These are the currently available tag checkers
meta_tags = {"title":{'column':1, 'min_char':0, 'max_char':50},
             "description":{'column':2, 'min_char':50, 'max_char':300}}
mapping_of_results = {}

for tag in meta_tags:
    # These are what the headers of the outputted xlsx will be, along with the output printed after running this cell.
    cols = ["Match Result", "Status Code","Url", "Expected "+tag, "actual "+tag, "SEO Check"]

    # This will be the ouputted table that will hold all of the results for that tag.
    #It starts empty to have a container to put results in.
    list_of_results = pd.DataFrame(columns=cols)
    
    min_seo = meta_tags[tag]['min_char']
    max_seo = meta_tags[tag]['max_char']

    for i in range(1, len(check_sheet.rows)):
        '''For every row in the input data, check to see that the meta tag 1) exists 2) is what was desired'''
        
        # Clear the results to be sure they don't carry over row to row
        seo_result = ""
        result = ""
        actual_tag = ""

        # Get data from the inputted file and add https to the front of a url if not full path
        url_containing_tag = add_https_if_none(to_check_sheet.cell(i, 0).value.strip())
        expected_tag = to_check_sheet.cell(i, meta_tags[tag]['column']).value.strip()

        try:
            url_status_code = requests.get(url_containing_tag).status_code

            if url_status_code in [200, 301]:
                if expected_tag != "":
                    actual_tag = return_tag(url_containing_tag, tag)
                    

                    if actual_tag == expected_tag:
                        result = "OK"
                    else:
                        result = "Expected and actual meta "+str(tag)+" do not match"
                    
                    if len(expected_tag) >= min_seo and len(expected_tag) <= max_seo:
                        seo_result = "OK"
                    elif len(expected_tag) < min_seo:
                        seo_result = "Too short, should be >= "+str(min_seo)
                    else:
                        seo_result = "Too long, should be <= "+str(max_seo)
                        
                else:
                    result = "n/a"
                    seo_result = "n/a"
            else: 
                result = "Error when accessing url containing meta tag "+str(tag)+". See status code."
        except:
            url_status_code = "Error"
            result = "url could not be accessed, double check: "+str(to_check_sheet.cell(i, 0).value.strip())


        # Append the result to a dataframe for output later
        list_of_results.loc[i] = [result, url_status_code, url_containing_tag, expected_tag, actual_tag, seo_result]

    print(list_of_results)
    mapping_of_results[tag] = list_of_results

                                        Match Result Status Code  \
1                                                 OK         200   
2                                                 OK         200   
3                                                n/a         200   
4                                                 OK         200   
5        Expected and actual meta title do not match         200   
6                                                n/a         200   
7  url could not be accessed, double check: https...       Error   

                                                 Url  \
1                        https://www.masterlock.com/   
2                https://www.masterlock.com/about-us   
3  https://www.masterlock.com/personal-use/school...   
4     https://www.masterlock.com/service-and-support   
5  https://www.masterlock.com/service-and-support...   
6  https://www.masterlock.com/service-and-support...   
7                                 https://www.badurl   

      

## Create Result Output File

After running the cell below, the results gotten from checking canonicals will be placed in an xlsx with the current timestamp in the title and then outputted to the __Results__ folder.

In [22]:
# Run to output the dataframe as an xlsx file in the 'Results' folder

# Time stamped file. In form yyyy-mm-dd_hour-minute
OUTPUT_FILE = META_TAGS_OUTPUT_FOLDER + 'meta-tag-results_'+ datetime.now().strftime("%Y-%m-%d_%H-%M") + '.xlsx'

writer = pd.ExcelWriter(OUTPUT_FILE, engine='xlsxwriter',)
for tag in meta_tags:
    '''For each of the tags, create a new sheet with its results and add it to the outputted file.'''
    mapping_of_results[tag].to_excel(writer, sheet_name=tag, index=False)
writer.save()